In [ ]:
!pip install torchvision==0.16.0
!pip install seaborn
# 安装后点击上方 Restart the kernal

In [ ]:
# 步骤一：导入相关库
import os
import torch
import torch.nn as nn
import torch.utils.data as Data
import torchvision
import torchvision.transforms as transforms
from torch_npu.contrib import transfer_to_npu
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

In [ ]:
# 步骤二：加载数据集
# 2.1 设置超参数
EPOCH = 10
BATCH_SIZE = 64
LR = 0.003
DOWNLOAD_CIFAR = False
DEVICE = 'npu'

# 2.2 数据预处理
transform = transforms.Compose([
    ...  # 请在此处编写代码：ToTensor 和 Normalize（均值与方差）
])

# 2.3 检查是否需要下载数据集
if not(os.path.exists('./cifar/')) or not os.listdir('./cifar/'):
    DOWNLOAD_CIFAR = True

# 2.4 加载训练集和测试集
train_data = torchvision.datasets.CIFAR10(
    root='./cifar/',
    train=True,
    transform=transform,
    download=DOWNLOAD_CIFAR
)
test_data = torchvision.datasets.CIFAR10(
    root='./cifar/',
    train=False,
    transform=transform,
    download=DOWNLOAD_CIFAR
)

# 2.5 封装数据加载器
train_loader = ...  # 请在此处编写代码
test_loader = ...   # 请在此处编写代码

In [ ]:
# 步骤三：CNN模型设计
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv = nn.Sequential(
            ...  # 请在此处编写代码：Conv2d + BatchNorm2d + ReLU + MaxPool2d
        )
        self.fc = nn.Sequential(
            ...  # 请在此处编写代码：Flatten + Linear + ReLU + Dropout + Linear
        )

    def forward(self, x):
        ...  # 请在此处编写代码

# 3.2 实例化模型并迁移到设备
cnn = ...  # 请在此处编写代码
print(cnn)

In [ ]:
# 步骤四：设置损失函数与优化器
optimizer = ...  # 请在此处编写代码：Adam优化器
criterion = ...  # 请在此处编写代码：交叉熵损失

In [ ]:
# 步骤五：训练模型
train_losses = []
test_accuracies = []

# 5.1 定义训练函数
def train(epoch):
    ...  # 请在此处编写代码：切换到训练模式（model.train()）
    epoch_loss = 0.0
    for batch_idx, (inputs, targets) in enumerate(tqdm(train_loader, desc=f"Training Epoch {epoch+1}")):
        ...  # 请在此处编写代码：将 inputs 和 targets 迁移到设备（to(DEVICE)）

        ...  # 请在此处编写代码：清除梯度（optimizer.zero_grad）
        ...  # 请在此处编写代码：前向传播（model(inputs)）
        ...  # 请在此处编写代码：计算损失（criterion(outputs, targets)）
        ...  # 请在此处编写代码：反向传播（loss.backward）
        ...  # 请在此处编写代码：参数更新（optimizer.step）

        ...  # 请在此处编写代码：累计损失（epoch_loss += loss.item()）

        # 可选：每 N 批次打印中间 loss
        if batch_idx % 100 == 0:
            ...  # 请在此处编写代码：打印当前 batch 的平均 loss

    ...  # 请在此处编写代码：计算 epoch 平均 loss
    train_losses.append(avg_epoch_loss)
    print(f"Epoch [{epoch+1}] Average Train Loss: {avg_epoch_loss:.4f}")

# 5.2 定义测试函数（返回 acc，预测标签，真实标签）
def test():
    ...  # 请在此处编写代码：切换模型为评估模式
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    with ...:  # 请在此处编写代码：关闭梯度计算（torch.no_grad）
        for inputs, targets in tqdm(test_loader, desc="Testing"):
            ...  # 请在此处编写代码：数据迁移到DEVICE
            ...  # 请在此处编写代码：前向传播
            _, predicted = ...  # 请在此处编写代码：获取预测标签（outputs.max(dim=1)）
            ...  # 请在此处编写代码：更新 total 和 correct 数量
            ...  # 请在此处编写代码：保存预测和真实标签到 all_preds 和 all_labels

    acc = ...  # 请在此处编写代码：计算准确率（百分比形式）
    pred_y = torch.cat(all_preds)
    true_y = torch.cat(all_labels)
    return acc, pred_y, true_y

# 5.3 开始训练过程
os.makedirs('./output', exist_ok=True)

for epoch in range(EPOCH):
    ...  # 请在此处编写代码：调用 train 函数
    acc, _, _ = ...  # 请在此处编写代码：调用 test 函数
    ...  # 请在此处编写代码：保存本轮测试准确率
    print(f"Test Accuracy after Epoch [{epoch+1}]: {acc:.4f}")

In [ ]:
# 步骤六：模型评估
# 6.1 最后一次测试
final_acc, pred_y, true_y = test()
print(f"\nFinal Test Accuracy: {final_acc:.2f}%")

# 6.2 绘制训练loss变化曲线
plt.figure()
plt.plot(range(1, EPOCH+1), train_losses, marker='o', label='Train Loss')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss over Epochs")
plt.legend()
plt.grid(True)
plt.savefig('./output/train_loss_curve.png')
plt.show()

# 6.3 绘制测试准确率变化曲线
plt.figure()
plt.plot(range(1, EPOCH+1), test_accuracies, marker='s', color='orange', label='Test Accuracy')
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Test Accuracy over Epochs")
plt.legend()
plt.grid(True)
plt.savefig('./output/test_accuracy_curve.png')
plt.show()

# 6.4 绘制混淆矩阵
conf_mat = confusion_matrix(true_y.numpy(), pred_y.numpy())

plt.figure(figsize=(8, 6))
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues')
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix on CIFAR-10 Test Set")
plt.savefig('./output/confusion_matrix.png')
plt.show()

# 6.5 打印前10个预测结果
print('Prediction:', pred_y[:10].numpy())
print('Actual:    ', true_y[:10].numpy())

In [ ]:
# 步骤七：保存模型与结果
# 请在此处编写代码：保存模型

In [ ]:
# 步骤八：推理结果可视化
# 8.1 CIFAR10 类别标签
classes = ['plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']

# 8.2 可视化推理函数
def visualize_cifar10_predictions(model, test_loader, device='cpu', num_images=10):
    """
    可视化模型在 CIFAR-10 测试集上的部分预测结果（RGB 彩色图）
    """
    ...  # 请在此处编写代码：将模型设置为 eval 模式
    shown = 0
    plt.figure(figsize=(15, 6))

    with torch.no_grad():
        for inputs, targets in test_loader:
            ...  # 请在此处编写代码：将 inputs 和 targets 迁移到指定设备
            outputs = ...  # 请在此处编写代码：前向传播获得模型输出
            _, predicted = ...  # 请在此处编写代码：获取预测结果

            for i in range(inputs.size(0)):
                if shown >= num_images:
                    break

                img = ...  # 请在此处编写代码：提取第 i 张图像并反归一化，还原为 (32, 32, 3)
                true_label = ...  # 请在此处编写代码：获取真实类别名称
                pred_label = ...  # 请在此处编写代码：获取预测类别名称

                plt.subplot(2, (num_images + 1) // 2, shown + 1)
                plt.imshow(img)
                plt.title(f"Pred: {pred_label} | True: {true_label}")
                plt.axis('off')
                shown += 1

            if shown >= num_images:
                break

    plt.suptitle("CIFAR-10 Inference Results on Test Images")
    plt.tight_layout()
    plt.savefig('./output/cifar10_test_predictions.png')
    plt.show()

# 8.3 执行推理可视化
visualize_cifar10_predictions(cnn, test_loader, device=DEVICE, num_images=10)